# Using AstraPy with vector database

Work with AstraPy

Make sure you have a Database and get ready to upload the Secure Connect Bundle and supply the Token string
(see [Pre-requisites](https://cassio.org/start_here/#vector-database) on cassio.org for details. Remember you need a **custom Token** with role [Database Administrator](https://awesome-astra.github.io/docs/pages/astra/create-token/)).

### Set up your python environment




In [ ]:
!curl -H 'Cache-Control: no-cache, no-store'  https://raw.githubusercontent.com/synedra/astra_vector_examples/main/notebook/astrapy.zip --output astrapy.zip
%pip install astrapy.zip


### Import needed libraries



In [2]:
from astrapy.base import AstraClient, http_methods
from astrapy.serverless import AstraJsonClient
from astrapy.vector import AstraVectorClient
from astrapy.ops import AstraOps
import os




### Astra Setup



### Astra Variables

In [ ]:
# Input your database keyspace name:
ASTRA_DB_ID = input('Your Astra DB id: ')

In [ ]:
# Input your database keyspace name:
ASTRA_DB_KEYSPACE = input('Your Astra DB Keyspace name (e.g. vector): ')

In [ ]:
# Input your Astra DB token string, the one starting with "AstraCS:..."
from getpass import getpass
ASTRA_DB_APPLICATION_TOKEN = getpass('Your Astra DB Token ("AstraCS:..."): ')

In [ ]:
# Input your Astra DB token string, the one starting with "AstraCS:..."
from getpass import getpass
ASTRA_DB_REGION = getpass('Your Astra DB Region: ')

In [2]:
# Input your Astra DB token string, the one starting with "AstraCS:..."
from getpass import getpass
TEST_COLLECTION_NAME = getpass('Collection name for tutorial: (e.g. "vector_test") ')

In [ ]:
### Client setup
astra_client = AstraClient(
        astra_database_id=ASTRA_DB_ID,
        astra_database_region=ASTRA_DB_REGION,
        astra_application_token=ASTRA_DB_APPLICATION_TOKEN,
    )

vector_client = AstraVectorClient(astra_client=astra_client)
test_namespace = vector_client.namespace(ASTRA_DB_KEYSPACE)
test_collection = vector_client.namespace(ASTRA_DB_KEYSPACE).collection(TEST_COLLECTION_NAME)

# Create and view collections

In [ ]:
res = test_namespace.create_vector_collection(name=TEST_COLLECTION_NAME, size=5)
res = test_namespace.get_collections()
print(res)

# Insert one document

In [ ]:
json_query = {
        "_id": "4",
        "name": "Coded Cleats Copy",
        "description": "ChatGPT integrated sneakers that talk to you",
        "$vector": [0.25, 0.25, 0.25, 0.25, 0.25],
    }

res = test_collection.create(document=json_query)
print(res)

# Insert multiple documents

In [ ]:
json_query = [
        {
            "_id": "1",
            "name": "Coded Cleats",
            "description": "ChatGPT integrated sneakers that talk to you",
            "$vector": [0.1, 0.15, 0.3, 0.12, 0.05],
        },
        {
            "_id": "2",
            "name": "Logic Layers",
            "description": "An AI quilt to help you sleep forever",
            "$vector": [0.45, 0.09, 0.01, 0.2, 0.11],
        },
        {
            "_id": "3",
            "name": "Vision Vector Frame",
            "description": "Vision Vector Frame - A deep learning display that controls your mood",
            "$vector": [0.1, 0.05, 0.08, 0.3, 0.6],
        },
    ]

res = test_collection.insert_many(documents=json_query)
print(res)